In [81]:
import torch
import torchvision
import tqdm
import matplotlib.pyplot as plt

GPU

In [82]:
# 使用 GPU
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

加载数据

In [83]:
# 数据预处理：标准化图像数据，使得灰度数据在-1到+1之间
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5,), (0.5,))])

# 下载Fashion-MNIST训练集数据，并构建训练集数据载入器trainloader,每次从训练集中载入64张图片，每次载入都打乱顺序
trainset = torchvision.datasets.FashionMNIST('dataset/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 下载Fashion-MNIST测试集数据，并构建测试集数据载入器trainloader,每次从测试集中载入64张图片，每次载入都打乱顺序
testset = torchvision.datasets.FashionMNIST('dataset/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

深度卷积神经网路

In [84]:
class alexnet(torch.nn.Module):
    def __init__(self):
        super(alexnet, self).__init__()
        self.layout1 = torch.nn.Sequential(torch.nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), torch.nn.ReLU(),
                                 torch.nn.MaxPool2d(kernel_size=3, stride=2))
        self.layout2 = torch.nn.Sequential(torch.nn.Conv2d(96, 256, kernel_size=5, padding=2), torch.nn.ReLU(),
                                 torch.nn.MaxPool2d(kernel_size=3, stride=2))
        self.layout3 = torch.nn.Sequential(torch.nn.Conv2d(256, 384, kernel_size=3, padding=1), torch.nn.ReLU())
        self.layout4 = torch.nn.Sequential(torch.nn.Conv2d(384, 384, kernel_size=3, padding=1), torch.nn.ReLU())
        self.layout5 = torch.nn.Sequential(torch.nn.Conv2d(384, 256, kernel_size=3, padding=1), torch.nn.ReLU(),
                                           torch.nn.MaxPool2d(kernel_size=3, stride=2))
        self.layout6 = torch.nn.Flatten()
        self.layout7 = torch.nn.Sequential(torch.nn.Linear(6400, 4096), torch.nn.ReLU(), torch.nn.Dropout(0.5),
                                           torch.nn.Linear(4096, 4096), torch.nn.ReLU(), torch.nn.Dropout(0.5),
                                           torch.nn.Linear(4096, 10), torch.nn.ReLU(), torch.nn.Dropout(0.5))

    
    def forward(self, x):
        x = self.layout1(x)
        x = self.layout2(x)
        x = self.layout3(x)
        x = self.layout4(x)
        x = self.layout5(x)
        x = self.layout6(x)
        x = self.layout7(x)
        return x

训练

In [85]:
net = alexnet()
net = net.to(device=try_gpu())

net.train()

cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

loss_data = {
    "epoch" : [],
    "loss" : []
}

i = 0
for epoch in tqdm.tqdm(range(5)):
    for images, labels in trainloader:
        images = images.to(device=try_gpu())
        labels = labels.to(device=try_gpu())
        y_predict = net(images)
        optimizer.zero_grad()
        loss = cost( y_predict, labels )
        loss_data["epoch"].append(i)
        loss_data["loss"].append(float(loss))
        loss.backward()
        optimizer.step()
        i = i + 1

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Given input size: (256x2x2). Calculated output size: (256x0x0). Output size is too small

Loss

In [ ]:
print(loss_data)
plt.plot(loss_data["epoch"], loss_data["loss"])
plt.show()

准确率

In [ ]:
net.eval()
yhat = []
epoch = 0
accuracy_sum = 0
for images, labels in testloader:
    images = images.to(device=try_gpu())
    labels = labels.to(device=try_gpu())
    y_predict = net(images)
    y_predict = torch.argmax(y_predict, dim = 1)
    accuracy = torch.eq(labels, y_predict).float().mean()
    accuracy_sum = accuracy_sum + accuracy
    epoch = epoch + 1
print("平均精准度：%f" % (accuracy_sum / epoch))